### Testing 5 different transformer models with 3 seeds each
### Task: Argument Stance Classification (Support/Oppose)

In [7]:
# Libraries
import os
import numpy as np
import pandas as pd
from datasets import Dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
import torch
from tqdm.auto import tqdm
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Google Colab or not
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    path = "/content/drive/MyDrive/multimodal-argmining"
    os.chdir(path)
    print(f"Loading data from Google Drive: {path}")
else:
    path = "C:/Users/diego/Desktop/Master Neuro/M2/Intership_NLP/multimodal-argmining"
    os.chdir(path)
    print(f"Loading data locally from: {path}")


# GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU ready:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("No GPU detecting, using CPU.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading data from Google Drive: /content/drive/MyDrive/multimodal-argmining
GPU ready: Tesla T4


In [8]:
# WE define our Models to tests and the seeds
MODELS = [
    "roberta-base",
    "microsoft/deberta-v3-base",
    "vinai/bertweet-base",
    "cardiffnlp/twitter-roberta-base",
    "bert-base-uncased",
    "microsoft/deberta-base"
]

#Seeds
SEEDS = [42, 123, 456]


print("\nModels to test:")
for i, model in enumerate(MODELS, 1):
    print(f"  {i}. {model}")



Models to test:
  1. roberta-base
  2. microsoft/deberta-v3-base
  3. vinai/bertweet-base
  4. cardiffnlp/twitter-roberta-base
  5. bert-base-uncased
  6. microsoft/deberta-base


In [9]:
os.getcwd()

'/content/drive/MyDrive/multimodal-argmining'

In [10]:
#Load Dataset
train_path = f"{path}/data/train.csv"
dev_path   = f"{path}/data/dev.csv"
test_path  = f"{path}/data/test.csv"

df_train = pd.read_csv(train_path)
df_dev   = pd.read_csv(dev_path)
df_test  = pd.read_csv(test_path)


# Map labels to ints
label2id = {"oppose": 0, "support": 1}
for df in [df_train, df_dev, df_test]:
    df["label"] = df["stance"].map(label2id)

print(df_train["label"].value_counts())
df_train.head()



dataset_train = Dataset.from_pandas(df_train[["tweet_text", "label"]])
dataset_dev   = Dataset.from_pandas(df_dev[["tweet_text", "label"]])
dataset_test  = Dataset.from_pandas(df_test[["tweet_text", "label"]])


label
0    1095
1     719
Name: count, dtype: int64


In [11]:

# Tokenization Function for each model
def tokenize_dataset(dataset, tokenizer, max_length=105):

    def tokenize_batch(batch):
        return tokenizer(batch["tweet_text"],padding="max_length",truncation=True,max_length=max_length)

    tokenized = dataset.map(tokenize_batch, batched=True)

    tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    return tokenized


In [12]:
# We define metrics Function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)

    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="macro")
    precision = precision_score(labels, preds, average="macro")
    recall = recall_score(labels, preds, average="macro")

    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall,
    }

In [13]:
# Training Function
def train_and_evaluate(model_name, seed, train_dataset, dev_dataset,test_dataset):

    # Set seed!
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    print(f"Training: {model_name} | Seed: {seed}")
    print(f"{'='*60}")

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    print(f"Tokenizer loaded for {model_name}...")

    # Tokenize datasets with model tokenizer
    train_dataset_tok = tokenize_dataset(train_dataset, tokenizer, 105)
    dev_dataset_tok = tokenize_dataset(dev_dataset, tokenizer, 105)
    test_dataset_tok = tokenize_dataset(test_dataset, tokenizer, 105)
    print(f"Tokenization complete")


    # Load model
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    print(f"Model Loaded: {model_name}.")


    # Training arguments
    training_args = TrainingArguments(
        output_dir=f"./temp_models/{model_name.replace('/', '_')}_seed{seed}",
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=5,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        save_total_limit=1,
        report_to="none",
        logging_steps=10
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset_tok,
        eval_dataset=dev_dataset_tok,
        processing_class=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    )

    # Train
    print(f"\n Starting training...")
    trainer.train()

    # Predictions on TEST set
    print(f"\n Getting predictions on TEST set...")
    predictions_output = trainer.predict(test_dataset_tok)
    y_pred = np.argmax(predictions_output.predictions, axis=1)
    y_true = predictions_output.label_ids



    # Evaluate
    print(f"\n Evaluating on test set...")
    eval_results = trainer.evaluate(test_dataset_tok)

    # Extract metrics
    results = {
        "model": model_name,
        "seed": seed,
        "accuracy": eval_results["eval_accuracy"],
        "precision": eval_results["eval_precision"],
        "recall": eval_results["eval_recall"],
        "f1": eval_results["eval_f1"],
        "loss": eval_results["eval_loss"],
        "y_true": y_true,
        "y_pred": y_pred
    }

    print(f"\n Results for {model_name} (seed {seed}):")
    print(f"   Accuracy:  {results['accuracy']:.4f}")
    print(f"   Precision: {results['precision']:.4f}")
    print(f"   Recall:    {results['recall']:.4f}")
    print(f"   F1-Score:  {results['f1']:.4f}")

    # Clean up to save memory
    del model
    del trainer
    del train_dataset_tok
    del dev_dataset_tok
    del test_dataset_tok

    torch.cuda.empty_cache() if torch.cuda.is_available() else None

    return results

In [14]:
# We run the Experiments
print("STARTING EXPERIMENT")
print("="*60)

all_results = []
total_experiments = len(MODELS) * len(SEEDS)
current_experiment = 0

for model_name in MODELS:
    print(f"# MODEL: {model_name}")

    for seed in SEEDS:
        current_experiment += 1
        print(f"\n[Experiment {current_experiment}/{total_experiments}]")
        results = train_and_evaluate(
            model_name=model_name,
            seed=seed,
            train_dataset=dataset_train,
            dev_dataset=dataset_dev,
            test_dataset=dataset_test)


        all_results.append(results)


print("EXPERIMENT COMPLETED")

STARTING EXPERIMENT
# MODEL: roberta-base

[Experiment 1/18]
Training: roberta-base | Seed: 42


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...


 Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.514100,0.256492,0.885000,0.869463,0.898616,0.854115
2,0.271800,0.222866,0.905000,0.892991,0.917857,0.878600
3,0.146000,0.321819,0.885000,0.877656,0.873371,0.883238
4,0.070900,0.467914,0.890000,0.872552,0.919137,0.852227



 Getting predictions on TEST set...



 Evaluating on test set...



 Results for microsoft/deberta-base (seed 456):
   Accuracy:  0.8300
   Precision: 0.8248
   Recall:    0.8390
   F1-Score:  0.8268
EXPERIMENT COMPLETED


In [15]:
# Results
results_df = pd.DataFrame(all_results)

# Per model we calculate the mean and std on each metric
model_stats = results_df.groupby('model').agg({
    'accuracy': ['mean', 'std'],
    'precision': ['mean', 'std'],
    'recall': ['mean', 'std'],
    'f1': ['mean', 'std']
}).round(4)

# Format
model_stats.columns = ['_'.join(col).strip() for col in model_stats.columns.values]
model_stats = model_stats.reset_index()
model_stats.columns = [
    'Model',
    'Accuracy_Mean', 'Accuracy_Std',
    'Precision_Mean', 'Precision_Std',
    'Recall_Mean', 'Recall_Std',
    'F1_Mean', 'F1_Std'
]

# Sort by F1 score
model_stats = model_stats.sort_values('F1_Mean', ascending=False).reset_index(drop=True)
print(model_stats)


                             Model  Accuracy_Mean  Accuracy_Std  \
0                     roberta-base         0.8500        0.0033   
1              vinai/bertweet-base         0.8311        0.0117   
2        microsoft/deberta-v3-base         0.8211        0.0236   
3  cardiffnlp/twitter-roberta-base         0.8167        0.0115   
4                bert-base-uncased         0.8133        0.0115   
5           microsoft/deberta-base         0.8100        0.0219   

   Precision_Mean  Precision_Std  Recall_Mean  Recall_Std  F1_Mean  F1_Std  
0          0.8448         0.0038       0.8600      0.0042   0.8471  0.0035  
1          0.8262         0.0107       0.8404      0.0104   0.8279  0.0115  
2          0.8275         0.0158       0.8416      0.0182   0.8198  0.0229  
3          0.8174         0.0066       0.8315      0.0060   0.8144  0.0104  
4          0.8090         0.0052       0.8208      0.0038   0.8094  0.0097  
5          0.8109         0.0161       0.8250      0.0169   0.8078  

In [16]:
# Exporting Confusion Matrix

# Labels
labels = ["oppose", "support"]

# Path Output
os.makedirs(f"{path}/experiments/text/Performance/", exist_ok=True)
output_dir = f"{path}/experiments/text/Performance/"

for model_name, group in results_df.groupby("model"):

    # Concatenate all the y_true and y_pred per model
    y_true_combined = np.concatenate(group["y_true"].values)
    y_pred_combined = np.concatenate(group["y_pred"].values)

    # Confusion Matrix
    cm = confusion_matrix(y_true_combined, y_pred_combined, labels=[0, 1])

    #F1-Score
    f1_macro = f1_score(y_true_combined, y_pred_combined, average='macro')

    # Plot
    plt.figure(figsize=(6, 5))
    sns.heatmap(
        cm,
        annot=True,
        fmt="d",
        cmap="Blues",
        cbar=False,
        xticklabels=labels,
        yticklabels=labels
    )

    plt.title(f"Confusion Matrix - {model_name}\nF1 (Macro): {f1_macro:.4f}")
    plt.xlabel("Prediction")
    plt.ylabel("True Label")

    # Guardar
    safe_model_name = model_name.replace("/", "_").replace("\\", "_")
    save_path = os.path.join(output_dir, f"confusion_matrix_{safe_model_name}.jpg")
    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close()

    print(f"Confusion Matrix saved for {model_name}: {save_path}")


Confusion Matrix saved for bert-base-uncased: /content/drive/MyDrive/multimodal-argmining/experiments/text/Performance/confusion_matrix_bert-base-uncased.jpg
Confusion Matrix saved for cardiffnlp/twitter-roberta-base: /content/drive/MyDrive/multimodal-argmining/experiments/text/Performance/confusion_matrix_cardiffnlp_twitter-roberta-base.jpg
Confusion Matrix saved for microsoft/deberta-base: /content/drive/MyDrive/multimodal-argmining/experiments/text/Performance/confusion_matrix_microsoft_deberta-base.jpg
Confusion Matrix saved for microsoft/deberta-v3-base: /content/drive/MyDrive/multimodal-argmining/experiments/text/Performance/confusion_matrix_microsoft_deberta-v3-base.jpg
Confusion Matrix saved for roberta-base: /content/drive/MyDrive/multimodal-argmining/experiments/text/Performance/confusion_matrix_roberta-base.jpg
Confusion Matrix saved for vinai/bertweet-base: /content/drive/MyDrive/multimodal-argmining/experiments/text/Performance/confusion_matrix_vinai_bertweet-base.jpg


In [17]:

# Results to CSV
os.makedirs(f"{path}/experiments/text/Performance/", exist_ok=True)
output_file = f"{path}/experiments/text/Performance/model_comparison_results.csv"
model_stats.to_csv(output_file, index=False)
print(f"\nSummary results saved to: {output_file}")


Summary results saved to: /content/drive/MyDrive/multimodal-argmining/experiments/text/Performance/model_comparison_results.csv
